In [1]:
#| default_exp models

# models

**Set up**  
I love working with Pydantic models, but I'd like to add a few tweaks to make them even better to work with in Jupyter Notebooks

**The Goal**  
Update how Pydantic models are displayed in Jupyter Lab and Jupyter Notebook to make them more natural to work with in an IPython environment

**The Result**  
Subclassed Pydantic's BaseModel and add `_repr_html_` and `_repr_json_` methods


In [48]:
#| exporti 

from pydantic import BaseModel as PydanticBaseModel
from pydantic import ConfigDict
import logging
from json2html import json2html
from humble_chuck.delegation import delegates
from typing import *

## Displaying Objects in IPython

If Jupyter encounters an error while calling their display methods, it will try the next available option for displaying an object. For example in Jupyter Lab the default representation is JSON, but if there is an error displaying the JSON it will fall back on displaying with HTML. If the error persists, it will fall back on default \_\
`__repr__` or `__str__` methods. 

So for this exercise, we'll create a function that tries to dump a pydanitc model, but if anything goes wrong it will just issue a warning and pass. That way if there is an issue with our custom display, we'll just get Pydantics default display mechanism. Note that we're using `delegates` here, so the kwargs you'll see in the docs are not included in the signature. 

In [3]:
#| exports

@delegates(PydanticBaseModel.model_dump) 
def model_dump_for_display(
    model:PydanticBaseModel, #The model to by displayed
    **kwargs
):
    """Calls PydanticBaseModel.model_dump(), 
    but if there is an issue it raises a warning and passes to allow default representation.  

    Delegates kwargs to PydanticBaseModel.model_dump
    """
    kwargs['mode']='json'
    try:
        return model.model_dump(**kwargs)
    except Exception as e:
        logging.warning(e)
        pass

::: {.callout-note}
Because we are subclassing Pydantic's BaseModel, the docs shown here are taken from the parent class. 
:::

In [17]:
#| exports
class BaseModel(PydanticBaseModel):
    
    def _repr_json_(self):
        return model_dump_for_display(
            self,
            mode='json',
            **self.model_config.get('repr_kwargs',{})
        )

    def _repr_html_(self):
        
        return json2html.convert(
            model_dump_for_display(self,mode='json',**self.model_config.get('repr_kwargs',{}))
        )

In [32]:
from pydantic import Field,AliasGenerator
from typing import *
from datetime import date

### Example

Note that you can customize how objects get displayed in the model_config. Here we'll choose to display the object with aliases instead of field names.

In [42]:
class Project(BaseModel):
    """Model for capturing details about a construction project"""
    
    model_config = ConfigDict(
        alias_generator=AliasGenerator(            
            serialization_alias=lambda field_name: field_name.title().replace('_',' '),
        ),
        repr_kwargs={'by_alias':True} #<-- I can control how the model gets displayed in jupyter by provided kwargs to model_dump 
    )
    
    project_name: str = Field(..., description="Name of the construction project")
    start_date: date = Field(..., description="Date when the project started")
    end_date: Optional[date] = Field(default=None, description="Date when the project ended")
    description: Optional[str] = Field(default=None, description="Short description of the project")
    is_active: bool = Field(..., description="Indicates if the project is currently active")
    budget: Optional[Dict[str, float]] = Field(default=None, description="Budget with different risk assessments")
    employees: List[Dict[str, str]] = Field(..., description="List of employees working on the project")
    technologies_used: List[str] = Field(..., description="List of technologies used in the project")
    


In the docs, you'll see this example represented as HTML. In Jupyter Lab it get's displayed as interactive, collapsible JSON.

In [46]:
# Creating an instance of the model
example_project = Project(
    project_name="Highway Bridge Construction",
    start_date=date(2024, 1, 15),
    end_date=None,
    description="A large-scale project focused on building a new highway bridge.",
    is_active=True,
    budget={"conservative": 5_000_000, "base_line": 6_500_000, "worst_case": 8_000_000},
    employees=[
        {"name": "Alice Johnson", "roll": "Project Manager"},
        {"name": "Bob Smith", "roll": "Engineer"},
        {"name": "Clara Davis", "roll": "Site Supervisor"}
    ],
    technologies_used=["AutoCAD", "Revit", "MS Project"]
)

example_project

Project(project_name='Highway Bridge Construction', start_date=datetime.date(2024, 1, 15), end_date=None, description='A large-scale project focused on building a new highway bridge.', is_active=True, budget={'conservative': 5000000.0, 'base_line': 6500000.0, 'worst_case': 8000000.0}, employees=[{'name': 'Alice Johnson', 'roll': 'Project Manager'}, {'name': 'Bob Smith', 'roll': 'Engineer'}, {'name': 'Clara Davis', 'roll': 'Site Supervisor'}], technologies_used=['AutoCAD', 'Revit', 'MS Project'])